## Living with the Quotas

## Throttling

In [1]:
threads.forEach(function(d) {
    var messages = d.getMessages();
    doSomeThingWith(messages);
    Utilities.sleep(1000);x
})

ReferenceError: threads is not defined

Because this is such a common requirement in almost every script that ccesses rate-limited resources, it's a good idea to use a common library function for backoff:

Code for exponential backoff

The error messages can be internationalized for the kinds of errors that should be retried. An example of a quota error is given in Russian in the following snippet. To add your own language, just expand the list with the equivalent Apps Script error message.

## Spliting

Libraries

The properties service is a secure store, rather like the Window registry. Chapter 5 is dedicated entierlyto this service. Scripts each come with their own Properties service. You can mitigate the limitation on overall property size by using the Properties service of a library and passing it to the main script(s) that use it:

In [3]:
// library 
function getPropertyService() {
    return PropertiesSErvice.getScriptProperties();
}

// main script
function work() {
    var libraryProps = Libarary.getPropertyService();
    var mainProps = PropertiesService.getScriptProperties();
}

## Batching

Some scripts can be split so that they work on separate sections of the data being processed. Applying the preceding tehcnuque, you can use the Properties service of a common library to exchange progress information between scripts so one can pick up where the other left off.

Parallel Running

Similar to batching, a shared library Properties service can be used to orchestrate cooperating scripts, except in this case the scripts run simultaneously using MapReduce-type techniques to consolidate and pass data from one phase to another.

## Offloading 

Apps Script runs on Google Servers, but HTML service tasks can be orchestrated by Apps Scripts to run on the local client. Generally speaking, compute-intensive tasks will run more quickly when executed locally. The HTML service can asynchronously fire off Apps Script tasks to deal with processes that require interaction with Apps Script services.

Carefully offloading work to the HTML service is a way of completely circumventing many of the server-side limitations, but it's a complex topic that's beyond the VBA/Apps Script scope.

## Avoiding Service Calls

Many of the rate-limited services are related to fetching data externally or from Google services or APIs. In some cases, either the data will not have changed from one call to the next, or it is not critical that the very latest data be used. Apps Script provides a Cache service for rapid access to data, enabling the complete avoidance of quota-consuming service calls.

## Cache Service

The Cache service allos data to be stored in key/value pairs. It is subject to limitations in data size, but is not constrained by rate limits. As a result, it's often good practice to write serice or external data that might be needed by this script to cache, and to first check the cache to see if it's already there. Not only does this avoid service call quota consumption, but also cache is generally faster to access than Apps Script services or external data and doesn't need to be reprocessed.

Cache scopes

The selection of which cache to use depends on the data visibility required. The cache visibility is limited to the current script, but within that there are three different caches, described in Table 4-3.

In [4]:
var cache = CacheService.getScriptCache();

ReferenceError: CacheService is not defined

In [5]:
var data = getDataFromSomeService();
cache.put("mykey", data);

ReferenceError: getDataFromSomeService is not defined

A future call to this same script would first check to see if the data was already in the cache, and if not, make the service call, thus avoiding the service call if there had been one made recently:

In [6]:
var data = cache.get("mykey");
if (!data) {
    data = getDataFromSomeService();
    cache.put("mykey", data);
}

TypeError: Cannot read property 'get' of undefined

** Cache data expiry **

It's important that cache data expires in order to provoke a refresh from time to time. By default, the cache will expire after 10 minutes, but you can change that like so:

** Sharing cache **

Caches are scoped to a particular script, but there may be occasions when multiple scripts would want to benefit from each other's cached results. 

In [7]:
function getCacheService() {
    return CacheService.getScriptCache();
}

// main script
var cache = Library.getCacheService();
var data = cache.get("mykey");
if (!data) {
    data = getDataFromSomeService();
    cache.put("mykey", data);
}

ReferenceError: Library is not defined

# Ch 5 : The Properties Service

The first few chapters covered the Apps Script language, and how VBA concepts and structures could be translated into their JavaScript equivalents. 

VBA becomes useful when the shared libraries that allow access to the Office object model are referenced. In the same way, Apps Script uses Google Apps services to provide access to the object models associated with the apps it can extend. This book will concentrate on those that have Office equivalents, but will also touch on the Google versions of a few additional capabilities that VBA obtains by using other shared libraries. We'll also take a brief look at some service that VBA does not have, but that we need to use Apps Script effectively.

## APIs Versus Built-In Services

There are many Google APIs, and only a few of them have been instrumented as built-in services for Apps Script. Many of the services we'll look at here can be accessed from other languages through language-specific APIs, or as JSON REST APIs to be accessed by any language.

Under the hood, these built-in services use these APIs to process translated requets from Apps Script. It is perfectly possible to access them directly from Apps Script through their JSON APIs, and in fact in some cases not all of the underlying APIs' capabilities have been exposed in the Apps Script implementation, so there may be some occasions when this is required. The built-in services provide a convenient and well-integrated way of accessing these services, taking care of authentication and other complications behind the scenes.

The scripts family of services, though, are specific to Apps Script and can't generally be accessed through any other API. One such service is the Properties service, which provides a convenient place to store persistent data.

- DocumentProperties 
- ScriptProperties
- UserProperties

## The Registry Versus the Property Store

VBA normally uses the Windows registry to store persistent parameters, or more ambitions developers might use custom XML parts in an Office document. Parameters could be keys, application preferences, or some other data that needs to persist across sessions or be embedded in a document.

## Comparisons

In [8]:
var prop = PropertiesService.getScriptProperties();
prop.setProperty("startwars", JSON.stringify({
    "yoda":"frank oz",
    "leia":"carrie fisher"
}))

ReferenceError: PropertiesService is not defined

# Ch 6: Spreadsheets

- Class
  - Range

In [9]:
var range = sheet.getRange("a2:b4");

ReferenceError: sheet is not defined

In [13]:
var Properties = (function (properties) {
    
    'use strict';
    
    var APP_NAME = 'carriers';
    /**
     * get the service to use
     * @return {PropertiesService}
     */
    properties.getService = function () {
        return PropertiesService.getUserProperties();
    }
    
    /** 
     * get the properties for this app
     * @return {object|null} the properties
     */
    properties.get = function () {
        var prop = properties.getService().getProperty(APP_NAME);
        return prop ? JSON.parse(prop) : null;
    }
    
    /** set the properties for this app
     * @param {object} props the properties for this app
     * @return {object} the properties
     */
    properties.set = function (props) {
        properties.getService().setProperty(APP_NAME, JSON.stringify(props));
        return props;
    }
    
    return properties;
})(Properties || {});

In [ ]:
function setProperties() {
    Properties.set ( {
        carries: {
            sheet : "lookup",
            id : "1f4zuZZv2NiLuYSGB5j4ENFc6w~"
        },
        update: {
            sheet: "updateLookup",
            id: "dsafds"
        }
    })
}

In [ ]:
/**
 * create a structured table from a range of data
* @constructor ListObject
* @param {Range} range the data range
* @param {string} [name] the table name
* @param {boolean} [hasHeaders=true] whether the table has headers
* @return {ListObject} self
*/
function ListObject(range, name, hasHeaders) {
    var self = this;
    hasHeaders_ = fixOptional(hasHeaders, true);
}

// get the data from the range
var range_ = range;
var data_ = range_.getValues();
//generate a unique name for the table if none given
var name_ = name || 'table)'+ new Date().getTime().toString(16);
// get the header row
var headers_ = hasHeaders_ ? data_.shift() : null;

// get the header collection (using the VBA collection object)
var numCols_ = range_.getNumColumns();
self.ListColumns = new Collection();
self.ListRows = new Collection();

function reCalculate_ () {
    self.DataBodyRange = range_.offset(hasHeaders_ ?
                                      1: 0, 0, data_,.length, numCols_);
    self.HeaderRowRange = hasHeaders_ ?
        range_.offset(0, 0, 1, numCols_) : null;
    
    for (var i=0; i<numCols_; i++) {
        self.ListColumns.Add( {
            Index:i+1,
            Name:hasHeaders_ ?
        })
    }
}

# Ch7 : The Document App

# Ch8 : Gmail, Calendar, and Contacts Apps


In [14]:
/** 
 * get matching threads
 * @return {GmailThread[]} the threads
 */
threads.get = function () {
    return Utils.rateLimitExpBackoff(function () {
        // get all target threads
        return GamilApp.serach("flight data" +
            " after:"+"2015/08'/15" +
            " -label:"+"flight-data-processed" +
            " in:"+"inbox");
    })
}

ReferenceError: threads is not defined